## Map data
heatmap by counties, weights as pollution data

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pprint import pprint
import requests
import json
import copy
from typing import List
from urllib.request import urlretrieve
from urllib.parse import quote
import csv
import gmaps
import os
from config import g_key
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Import API key
from config import key
from config import email_key

In [5]:
# Import from .CSV
california_df = pd.read_csv("./csv_data/california_data.csv") 

In [6]:
california_df.head()

,state_code,state,county_code,county,site_number,parameter,sample_measurement,units_of_measure,latitude,longitude,date_local,time_local,date_gmt,time_gmt,method_type,method_code,method,date_of_last_change,cbsa_code
0,6,California,37,Los Angeles,9034,Acceptable PM2.5 AQI & Speciation Mass,NaN,Micrograms/cubic meter (LC),34.813034,-118.884819,2019-06-26,00:00,2019-06-26,08:00,Non-FRM,707,IMPROVE Module A with Cyclone Inlet-Teflon Fil...,2020-04-15,31080
1,6,California,37,Los Angeles,9034,Acceptable PM2.5 AQI & Speciation Mass,NaN,Micrograms/cubic meter (LC),34.813034,-118.884819,2019-04-30,00:00,2019-04-30,08:00,Non-FRM,707,IMPROVE Module A with Cyclone Inlet-Teflon Fil...,2020-03-04,31080
2,6,California,37,Los Angeles,9034,Acceptable PM2.5 AQI & Speciation Mass,NaN,Micrograms/cubic meter (LC),34.813034,-118.884819,2019-04-27,00:00,2019-04-27,08:00,Non-FRM,707,IMPROVE Module A with Cyclone Inlet-Teflon Fil...,2020-03-04,31080
3,6,California,37,Los Angeles,9034,Acceptable PM2.5 AQI & Speciation Mass,NaN,Micrograms/cubic meter (LC),34.813034,-118.884819,2019-04-24,00:00,2019-04-24,08:00,Non-FRM,707,IMPROVE Module A with Cyclone Inlet-Teflon Fil...,2020-03-04,31080
4,6,California,37,Los Angeles,9034,Acceptable PM2.5 AQI & Speciation Mass,NaN,Micrograms/cubic meter (LC),34.813034,-118.884819,2019-04-03,00:00,2019-04-03,08:00,Non-FRM,707,IMPROVE Module A with Cyclone Inlet-Teflon Fil...,2020-03-04,31080


In [7]:
# Make copy before filtering
california_filtered_df = california_df.copy()

In [8]:
# Select certain columns to use for mapping
california_filtered_df=california_filtered_df[['county','site_number','sample_measurement','latitude','longitude','time_local','date_gmt','time_gmt','date_of_last_change']]
california_filtered_df.head(20)

,county,site_number,sample_measurement,latitude,longitude,time_local,date_gmt,time_gmt,date_of_last_change
0,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-06-26,08:00,2020-04-15
1,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-04-30,08:00,2020-03-04
2,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-04-27,08:00,2020-03-04
3,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-04-24,08:00,2020-03-04
4,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-04-03,08:00,2020-03-04
5,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-02-11,08:00,2019-12-06
6,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-02-08,08:00,2019-12-06
7,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-02-05,08:00,2019-12-06
8,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-02-02,08:00,2019-12-06
9,Los Angeles,9034,NaN,34.813034,-118.884819,00:00,2019-01-06,08:00,2019-12-06


In [9]:
california_filtered_df=california_filtered_df.dropna()

In [10]:
california_filtered_df = california_filtered_df[(california_filtered_df[['sample_measurement']] > 0).all(axis=1)]
california_filtered_df.head()

,county,site_number,sample_measurement,latitude,longitude,time_local,date_gmt,time_gmt,date_of_last_change
11,Los Angeles,9034,1.4,34.813034,-118.884819,00:00,2019-04-06,08:00,2020-03-04
12,Los Angeles,9034,1.5,34.813034,-118.884819,00:00,2019-03-16,08:00,2020-03-04
13,Los Angeles,9034,2.3,34.813034,-118.884819,00:00,2019-06-29,08:00,2020-04-15
14,Los Angeles,9034,3.0,34.813034,-118.884819,00:00,2019-04-21,08:00,2020-03-04
15,Los Angeles,9034,2.2,34.813034,-118.884819,00:00,2019-03-28,08:00,2020-03-04


In [11]:
california_filtered_df.describe()

,site_number,sample_measurement,latitude,longitude
count,38401.000000,38401.000000,38401.000000,38401.000000
mean,2391.170647,11.212643,34.111205,-118.257149
std,2175.244013,9.246106,0.192668,0.254512
min,16.000000,0.100000,33.792360,-118.884819
25%,1103.000000,6.100000,34.066590,-118.528400
50%,1201.000000,10.000000,34.144350,-118.226880
75%,4004.000000,14.700000,34.199250,-118.175330
max,9034.000000,995.600000,34.813034,-117.850360


In [12]:
# Set Index
california_filtered_df=california_filtered_df.set_index('date_of_last_change')

# Sort Values
ca_dates = california_filtered_df.sort_values(by='date_of_last_change', axis=0).index.unique()

## Begin Heat Map. 

In [13]:
# Single out Lat,Lon values to combine into list
lat_cords = [i for i in california_filtered_df['latitude']]
lon_cords = [i for i in california_filtered_df['longitude']]
# Bring out humit values. Used for weights in gmaps
humid_weights = california_filtered_df['sample_measurement'].tolist()

In [14]:
# Go through two lists to create [(lat,lng)] for coordinate values. 
# Used in gmaps
coordinates = []
for i,j in zip(lat_cords, lon_cords):
    coords = (i,j)
    coordinates.append(coords)

In [15]:
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(coordinates, weights=humid_weights, 
                                 dissipating=False, max_intensity=.4,
                                 point_radius=.1)

#markers = gmaps.marker_layer(coordinates)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…

Edited to emphasize weights

In [16]:
figure_layout = {
    'width': '600px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Add Heatmap layer to map.
heat_layer = gmaps.heatmap_layer(coordinates, weights=humid_weights, 
                                 dissipating=False, max_intensity=.99,
                                 point_radius=.2)

#markers = gmaps.marker_layer(coordinates)
fig = gmaps.figure(layout=figure_layout)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='500px', margin='0 auto 0 auto', padding='1px', wi…